In [12]:
import pandas as pd     
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os
from general_pic_setup import setup_mpl_single2
from matplotlib.ticker import MultipleLocator, FixedLocator, FuncFormatter
import matplotlib as mpl

# 基础样式设置
setup_mpl_single2()
mpl.rcParams['axes.titlesize'] = 'medium'

script_dir = Path(os.getcwd())
data_dir = script_dir.parent / "data"
output_dir = data_dir / "2-2-Country_Policy_Trends"
output_dir.mkdir(parents=True, exist_ok=True)

df = pd.read_parquet(data_dir / "2-1-country_proportions.parquet")

country_col = 'REF_AREA'
year_col = 'TIME_PERIOD'

# 数据预处理
df[year_col] = pd.to_numeric(df[year_col], errors='coerce')
df = df.dropna(subset=[year_col])
df = df[(df[year_col] >= 2005) & (df[year_col] <= 2023)]
df = df.sort_values(year_col)

# ========== 政策-类别-国家映射（每个类别选多个知名国家）==========
policy_country_mapping = {
    "Electricity – non market-based instruments": {
        0: {  # 类别0（无或低水平）
            "countries": ["CHN", "ISL", "IDN", "MEX"],  # 中国、冰岛、印尼、墨西哥
            "color": "#FFC0CB",  # 红色系-浅色
            "label": "Low"
        },
        1: {  # 类别1（中等水平）
            "countries": ["GBR", "JPN", "KOR", "AUS", "CAN", "DEU", "FRA", "ITA", "ESP"],  # 英国、日本、韩国、澳大利亚、加拿大、德国、法国、意大利、西班牙
            "color": "#E64B35",  # 红色系-中色
            "label": "Medium"
        },
        2: {  # 类别2（高水平）
            "countries": ["TUR", "COL", "ISR", "CHL", "PER", "ROU"],  # 土耳其、哥伦比亚、以色列、智利、秘鲁、罗马尼亚
            "color": "#8B0000",  # 红色系-深色
            "label": "High"
        }
    },
    "International climate co-operation": {
        0: {  # 类别0
            "countries": ["AUS", "CHN", "JPN", "KOR", "DEU", "FRA", "ITA"],  # 澳大利亚、中国、日本、韩国、德国、法国、意大利
            "color": "#B2DFDB",  # 青绿色系-浅色
            "label": "Low"
        },
        1: {  # 类别1
            "countries": ["IND", "GBR", "ISL", "ISR", "ZAF"],  # 印度、英国、冰岛、以色列、南非
            "color": "#00A087",  # 青绿色系-中色
            "label": "Medium"
        },
        2: {  # 类别2
            "countries": ["IDN", "CHL", "COL", "SAU", "PER"],  # 印尼、智利、哥伦比亚、沙特、秘鲁
            "color": "#00695C",  # 青绿色系-深色
            "label": "High"
        }
    }
}

def lighten_color(color, amount=0.7):
    """将颜色变浅用于填充"""
    try:
        import matplotlib.colors as mcolors
        c = mcolors.to_rgb(color)
        c = tuple([c[i] + (1 - c[i]) * amount for i in range(3)])
        return c
    except Exception:
        return color

def setup_axes(ax, y_max_val, y_lim_min=0, years=None):
    """统一的轴设置"""
    # ========== 关键修改：设置X轴标注 ==========
    # 固定开始年份2005和结束年份2023
    if years is not None and 2005 in years and 2023 in years:
        # 获取2005和2023的索引位置
        start_idx = years.index(2005) if 2005 in years else 0
        end_idx = years.index(2023) if 2023 in years else len(years) - 1
        
        # 在中间均匀插入1个年份
        all_ticks = [2005, 2023]
        
        if len(years) > 2:
            # 计算中间年份的位置
            mid_idx = (start_idx + end_idx) // 2
            if 0 <= mid_idx < len(years):
                mid_year = years[mid_idx]
                if mid_year not in all_ticks:
                    all_ticks.append(mid_year)
        
        # 排序并设置刻度
        all_ticks = sorted(all_ticks)
        ax.set_xticks(all_ticks)
    else:
        # 如果2005或2023不在数据中，则使用原始的4年间隔
        ax.xaxis.set_major_locator(MultipleLocator(4))
    
    ax.set_xlim(2004.5, 2023.5)
    ax.tick_params(axis='x', rotation=45, pad=5.5)
    
    # ========== 关键修改：Y轴固定为0.0到0.4 ==========
    ax.yaxis.set_major_locator(FixedLocator([0.0, 0.4]))
    ax.yaxis.set_major_formatter(FuncFormatter(lambda x, p: f'{x:.1f}'))
    ax.set_ylim(0, 0.4)
    
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    ax.set_xlabel('Year', labelpad=8)
    ax.set_ylabel('Proportion', labelpad=8)

def plot_country_policy(country_code, policy_name, cluster_id, color, label, df_country):
    """绘制单个国家-政策图表"""
    if policy_name not in df_country.columns:
        print(f"      ⚠ 跳过 {country_code}：列不存在")
        return False
    
    if df_country[policy_name].isna().all():
        print(f"      ⚠ 跳过 {country_code}：无有效数据")
        return False
    
    # 计算Y轴范围
    y_min = df_country[policy_name].min()
    y_max = df_country[policy_name].max()
    
    if pd.isna(y_min) or pd.isna(y_max):
        print(f"      ⚠ 跳过 {country_code}：数据异常")
        return False
    
    # 创建图表
    fig, ax = plt.subplots(figsize=(10, 6))
    
    ax.plot(df_country[year_col], df_country[policy_name], 
            color=color, 
            linewidth=2.5,
            marker='^',
            markersize=10,
            markerfacecolor=lighten_color(color, amount=0.7),
            markeredgecolor=color, 
            markeredgewidth=1.8,
            zorder=10)
    
    setup_axes(ax, 0.4, years=df_country[year_col].tolist())
    
    plt.tight_layout()
    
    # 文件命名：国家_政策简称_类别标签
    policy_short = "Elec_NMBI" if "Electricity" in policy_name else "Climate_Coop"
    out_path = output_dir / f"{country_code}_{policy_short}_{label}.png"
    fig.savefig(out_path, dpi=300, bbox_inches='tight', pad_inches=0.1)
    plt.close()
    
    print(f"      ✓ {country_code} → {out_path.name}")
    return True

# ========== 主循环：为每个类别的所有国家生成图表 ==========
print(f"\n开始生成图表...\n")
total_plots = 0
total_policies = len(policy_country_mapping)

for policy_idx, (policy_name, clusters) in enumerate(policy_country_mapping.items(), 1):
    print(f"\n{'='*70}")
    print(f"[{policy_idx}/{total_policies}] 政策: {policy_name}")
    print(f"{'='*70}")
    
    for cluster_id, cluster_info in sorted(clusters.items()):
        countries = cluster_info['countries']
        color = cluster_info['color']
        label = cluster_info['label']
        
        print(f"\n  📊 类别 {cluster_id} ({label}) | 颜色: {color} | 国家数: {len(countries)}")
        print(f"    国家列表: {', '.join(countries)}")
        
        successful_plots = 0
        for country in countries:
            # 筛选国家数据
            df_country = df[df[country_col] == country].copy()
            
            if df_country.empty:
                print(f"      ⚠ 跳过 {country}：无数据")
                continue
            
            # 绘制图表
            if plot_country_policy(country, policy_name, cluster_id, color, label, df_country):
                total_plots += 1
                successful_plots += 1
        
        print(f"    ✅ 类别 {cluster_id} 完成: {successful_plots}/{len(countries)} 个国家")

print(f"\n{'='*70}")
print(f"✅ 完成！共生成 {total_plots} 个图表")
print(f"📁 保存位置: {output_dir}")
print(f"{'='*70}")

# 生成汇总报告
print(f"\n📋 类别汇总:")
for policy_name, clusters in policy_country_mapping.items():
    policy_short = "Elec_NMBI" if "Electricity" in policy_name else "Climate_Coop"
    print(f"\n  {policy_name}:")
    for cluster_id, cluster_info in sorted(clusters.items()):
        print(f"    - 类别{cluster_id} ({cluster_info['label']}): {len(cluster_info['countries'])} 个国家")


开始生成图表...


[1/2] 政策: Electricity – non market-based instruments

  📊 类别 0 (Low) | 颜色: #FFC0CB | 国家数: 4
    国家列表: CHN, ISL, IDN, MEX
      ✓ CHN → CHN_Elec_NMBI_Low.png
      ✓ ISL → ISL_Elec_NMBI_Low.png
      ✓ IDN → IDN_Elec_NMBI_Low.png
      ✓ MEX → MEX_Elec_NMBI_Low.png
    ✅ 类别 0 完成: 4/4 个国家

  📊 类别 1 (Medium) | 颜色: #E64B35 | 国家数: 9
    国家列表: GBR, JPN, KOR, AUS, CAN, DEU, FRA, ITA, ESP
      ✓ GBR → GBR_Elec_NMBI_Medium.png
      ✓ JPN → JPN_Elec_NMBI_Medium.png
      ✓ KOR → KOR_Elec_NMBI_Medium.png
      ✓ AUS → AUS_Elec_NMBI_Medium.png
      ✓ CAN → CAN_Elec_NMBI_Medium.png
      ✓ DEU → DEU_Elec_NMBI_Medium.png
      ✓ FRA → FRA_Elec_NMBI_Medium.png
      ✓ ITA → ITA_Elec_NMBI_Medium.png
      ✓ ESP → ESP_Elec_NMBI_Medium.png
    ✅ 类别 1 完成: 9/9 个国家

  📊 类别 2 (High) | 颜色: #8B0000 | 国家数: 6
    国家列表: TUR, COL, ISR, CHL, PER, ROU
      ✓ TUR → TUR_Elec_NMBI_High.png
      ✓ COL → COL_Elec_NMBI_High.png
      ✓ ISR → ISR_Elec_NMBI_High.png
      ✓ CHL → CHL_Elec_NMBI_High.png
  

In [ ]:
import pandas as pd     
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os
from general_pic_setup import setup_mpl_single2
from matplotlib.ticker import MultipleLocator, FixedLocator, FuncFormatter
import matplotlib as mpl
import matplotlib.cm as cm

# 基础样式设置
setup_mpl_single2()
mpl.rcParams['axes.titlesize'] = 'medium'

script_dir = Path(os.getcwd())
data_dir = script_dir.parent / "data"
output_dir = data_dir / "2-2-Country_Policy_Trends"
output_dir.mkdir(parents=True, exist_ok=True)

df = pd.read_parquet(data_dir / "2-1-country_proportions.parquet")

country_col = 'REF_AREA'
year_col = 'TIME_PERIOD'

# 数据预处理
df[year_col] = pd.to_numeric(df[year_col], errors='coerce')
df = df.dropna(subset=[year_col])
df = df[(df[year_col] >= 2005) & (df[year_col] <= 2023)]
df = df.sort_values(year_col)

# 选择7个不同大洲的知名国家（不指定颜色）
selected_countries_info = [
    ("CAN", "North America"),   # 加拿大 - 北美洲
    ("DEU", "Europe"),          # 德国 - 欧洲
    ("CHN", "Asia"),            # 中国 - 亚洲
    ("ARG", "South America"),   # 阿根廷 - 南美洲
    ("AUS", "Oceania"),         # 澳大利亚 - 大洋洲
    ("ZAF", "Africa"),          # 南非 - 非洲
    ("JPN", "Asia")             # 日本 - 亚洲
]

# 两个政策
policies = {
    "Electricity – non market-based instruments": "Electricity Policy",
    "International climate co-operation": "Climate Cooperation"
}

def plot_ridge_chart(policy_name, policy_label, selected_countries_info, df):
    """
    为一个政策创建山脊图，显示7个国家的趋势（使用原始数据，无样条插值）
    """
    # 创建图表
    fig, ax = plt.subplots(figsize=(3, 8))
    
    # 间距设置为0.1
    y_offset = 0.5
    
    # 使用colormap生成渐变色
    cmap = cm.get_cmap('Blues')
    n_countries = len(selected_countries_info)
    colors = [cmap(0.3 + 0.6 * i / (n_countries - 1)) for i in range(n_countries)]
    
    # 存储有效的国家
    valid_idx = 0
    
    # 绘制每个国家
    for idx, (country_code, continent) in enumerate(selected_countries_info):
        df_country = df[df[country_col] == country_code].copy()
        
        if df_country.empty or policy_name not in df_country.columns:
            print(f"    ⚠ 跳过 {country_code}：无数据")
            continue
        
        if df_country[policy_name].isna().all():
            print(f"    ⚠ 跳过 {country_code}：无有效数据")
            continue
        
        # 获取数据
        years = df_country[year_col].values
        values = df_country[policy_name].values
        
        # 移除NaN
        mask = ~np.isnan(values)
        years = years[mask]
        values = values[mask]
        
        if len(years) == 0:
            continue
        
        color = colors[idx]
        
        # Y轴基线位置（每个国家向上偏移）
        baseline = valid_idx * y_offset
        
        # 将数据限制在0到0.4之间
        values_clipped = np.clip(values, 0.0, 0.4)
        
        # 直接使用原始数据绘制（无插值）
        # 绘制填充区域
        # ax.fill_between(years, baseline, baseline + values_clipped,
        #                color=color, alpha=0.75, linewidth=0, zorder=valid_idx)
        
        # 绘制轮廓线
        ax.plot(years, baseline + values_clipped, 
               color=color, linewidth=1.8, alpha=0.95, zorder=valid_idx+100)
        
        # 添加国家标签（左侧显示国家代码）
        ax.text(2004, baseline + 0.02, f'{country_code}', 
               va='center', ha='right', fontsize=11, 
               color=color, weight='bold')
        
        # 添加大洲标签（右侧）
        ax.text(2023.8, baseline + 0.02, f'{continent}', 
               va='center', ha='left', fontsize=9, 
               color=color, alpha=0.8, style='italic')
        
        valid_idx += 1
    
    if valid_idx == 0:
        print(f"    ⚠ 没有有效数据可绘制")
        plt.close()
        return
    
    # 绘制垂直参考线（在中间年份）
    ref_year = 2014
    ax.axvline(x=ref_year, color='gray', linestyle='--', 
              linewidth=1.5, alpha=0.3, zorder=0)
    
    # 设置X轴：显示所有年份
    all_years = list(range(2005, 2024))
    ax.set_xlim(2003.5, 2024.5)
    ax.set_xticks(all_years)
    ax.set_xticklabels(all_years, fontsize=10)
    ax.set_xlabel('Year', fontsize=13, labelpad=10)
    ax.tick_params(axis='x', rotation=45)
    
    # Y轴范围根据间距和数据范围调整
    ax.set_ylim(0.0, valid_idx * y_offset + 0.4)
    ax.set_yticks([])
    ax.set_ylabel('Countries', fontsize=13, labelpad=10)
    
    # 隐藏边框
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_linewidth(1.5)
    
    plt.tight_layout()
    
    # 保存
    policy_short = "Electricity_NMBI" if "Electricity" in policy_name else "Climate_Cooperation"
    out_path = output_dir / f"{policy_short}_ridge_plot.png"
    fig.savefig(out_path, dpi=300, bbox_inches='tight', pad_inches=0.15)
    plt.close()
    
    print(f"  ✓ 山脊图已保存: {out_path.name} (包含 {valid_idx} 个国家)")

# 主循环：为两个政策生成山脊图
print(f"\n开始生成山脊图...\n")
print(f"选定的7个国家:")
for country_code, continent in selected_countries_info:
    print(f"  • {country_code} ({continent})")
print()

for policy_idx, (policy_name, policy_label) in enumerate(policies.items(), 1):
    print(f"\n{'='*70}")
    print(f"[{policy_idx}/{len(policies)}] 政策: {policy_name}")
    print(f"{'='*70}\n")
    
    plot_ridge_chart(policy_name, policy_label, selected_countries_info, df)

print(f"\n{'='*70}")
print(f"✅ 完成！共生成 {len(policies)} 个山脊图")
print(f"📁 保存位置: {output_dir}")
print(f"{'='*70}")


开始生成山脊图...

选定的7个国家:
  • CAN (North America)
  • DEU (Europe)
  • CHN (Asia)
  • ARG (South America)
  • AUS (Oceania)
  • ZAF (Africa)
  • JPN (Asia)


[1/2] 政策: Electricity – non market-based instruments

  ✓ 山脊图已保存: Electricity_NMBI_ridge_plot.png (包含 7 个国家)

[2/2] 政策: International climate co-operation



C:\Users\28274\AppData\Local\Temp\ipykernel_12692\2344208994.py:59: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('Blues')
C:\Users\28274\AppData\Local\Temp\ipykernel_12692\2344208994.py:59: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('Blues')


  ✓ 山脊图已保存: Climate_Cooperation_ridge_plot.png (包含 7 个国家)

✅ 完成！共生成 2 个山脊图
📁 保存位置: f:\Desktop\CAMPF_Supplementary\data\2-2-Country_Policy_Trends


In [17]:
import pandas as pd      
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os
from general_pic_setup import setup_mpl_single2
from matplotlib.ticker import MultipleLocator, FixedLocator, FuncFormatter
import matplotlib as mpl
import matplotlib.cm as cm
from matplotlib.patches import Rectangle
from matplotlib.collections import LineCollection

# 基础样式设置
setup_mpl_single2()
mpl.rcParams['axes.titlesize'] = 'medium'

script_dir = Path(os.getcwd())
data_dir = script_dir.parent / "data"
output_dir = data_dir / "2-2-Country_Policy_Trends"
output_dir.mkdir(parents=True, exist_ok=True)

df = pd.read_parquet(data_dir / "2-1-country_proportions.parquet")

country_col = 'REF_AREA'
year_col = 'TIME_PERIOD'

# 数据预处理
df[year_col] = pd.to_numeric(df[year_col], errors='coerce')
df = df.dropna(subset=[year_col])
df = df[(df[year_col] >= 2005) & (df[year_col] <= 2023)]
df = df.sort_values(year_col)

# 选择7个不同大洲的知名国家
selected_countries_info = [
    ("CAN", "North America"),   # 加拿大 - 北美洲
    ("DEU", "Europe"),          # 德国 - 欧洲
    ("CHN", "Asia"),            # 中国 - 亚洲
    ("ARG", "South America"),   # 阿根廷 - 南美洲
    ("AUS", "Oceania"),         # 澳大利亚 - 大洋洲
    ("ZAF", "Africa"),          # 南非 - 非洲
    ("JPN", "Asia")             # 日本 - 亚洲
]

# 两个政策
policies = {
    "Electricity – non market-based instruments": "Electricity Policy",
    "International climate co-operation": "Climate Cooperation"
}

# 统一线宽（除了曲线外的所有线条）
UNIFIED_LINEWIDTH = 1.0

def plot_ridge_chart(policy_name, policy_label, selected_countries_info, df):
    """
    为一个政策创建山脊图，显示7个国家的趋势
    紧密堆叠，每个曲线Y范围固定0-0.4，使用渐变色
    """
    # 创建图表 - 增大尺寸以容纳外部标签
    fig, ax = plt.subplots(figsize=(5, 9))
    
    # 每个国家占据固定的高度（Y值范围为0.4）
    box_height = 0.4
    
    # 使用colormap生成渐变色（从蓝色到红色）
    cmap = cm.get_cmap('coolwarm')
    
    # 存储有效的国家
    valid_idx = 0
    
    # 绘制每个国家
    for idx, (country_code, continent) in enumerate(selected_countries_info):
        df_country = df[df[country_col] == country_code].copy()
        
        if df_country.empty or policy_name not in df_country.columns:
            print(f"    ⚠ 跳过 {country_code}：无数据")
            continue
        
        if df_country[policy_name].isna().all():
            print(f"    ⚠ 跳过 {country_code}：无有效数据")
            continue
        
        # 获取数据
        years = df_country[year_col].values
        values = df_country[policy_name].values
        
        # 移除NaN
        mask = ~np.isnan(values)
        years = years[mask]
        values = values[mask]
        
        if len(years) == 0:
            continue
        
        # Y轴基线位置（紧密堆叠，每个国家占据0.4的高度）
        baseline = valid_idx * box_height
        
        # 将数据限制在0到0.4之间
        values_clipped = np.clip(values, 0.0, 0.4)
        
        # 添加边框 - 覆盖整个区域，黑色无透明度，统一线宽
        box_x = 2005
        box_width = 2023 - 2005
        box_y = baseline
        
        rect = Rectangle((box_x, box_y), box_width, box_height,
                        linewidth=UNIFIED_LINEWIDTH, edgecolor='black', 
                        facecolor='none', zorder=valid_idx-1)
        ax.add_patch(rect)
        
        # 创建渐变色曲线（不插值）
        points = np.array([years, baseline + values_clipped]).T.reshape(-1, 1, 2)
        segments = np.concatenate([points[:-1], points[1:]], axis=1)
        
        # 为每个线段分配颜色（从头到尾渐变）
        n_segments = len(segments)
        colors_gradient = [cmap(i / (n_segments - 1)) for i in range(n_segments)]
        
        lc = LineCollection(segments, colors=colors_gradient, linewidth=2.5, alpha=0.95, zorder=valid_idx+100)
        ax.add_collection(lc)
        
        # 添加国家标签（使用图坐标系，标在外面左侧，继续左移）
        ax.text(-0.12, (baseline + 0.2) / (7 * box_height), f'{country_code}', 
               transform=ax.transAxes,
               va='center', ha='right', fontsize=11, 
               color='#333333', weight='bold')
        
        # 添加大洲标签（使用图坐标系，标在外面右侧）
        ax.text(1.02, (baseline + 0.2) / (7 * box_height), f'{continent}', 
               transform=ax.transAxes,
               va='center', ha='left', fontsize=9, 
               color='#666666', alpha=0.8, style='italic')
        
        # 添加Y轴标注：0和0.4
        # 0.4标注在上边界线的上方
        ax.text(2004, baseline + 0.4, '0.4', 
               va='bottom', ha='right', fontsize=8, 
               color='black')
        
        # 特殊处理：最下面的山脊（第一个国家）的0标注放在上方
        if valid_idx == 0:
            ax.text(2004, baseline, '0', 
                   va='bottom', ha='right', fontsize=8, 
                   color='black')
        else:
            ax.text(2004, baseline, '0', 
                   va='top', ha='right', fontsize=8, 
                   color='black')
        
        valid_idx += 1
    
    if valid_idx == 0:
        print(f"    ⚠ 没有有效数据可绘制")
        plt.close()
        return
    
    # 设置X轴：严格限制在2005-2023范围内
    ax.set_xlim(2005, 2023)
    ax.set_xticks([2005, 2014, 2023])
    ax.set_xticklabels([2005, 2014, 2023], fontsize=11, rotation=60, ha='center')
    ax.set_xlabel('Year', fontsize=13, labelpad=3)
    
    # Y轴范围：紧密堆叠，从0到总高度
    ax.set_ylim(0, valid_idx * box_height)
    ax.set_yticks([])
    ax.set_ylabel('Countries', fontsize=13, labelpad=10)
    
    # 统一边框粗细和颜色为黑色，统一线宽
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_linewidth(UNIFIED_LINEWIDTH)
    ax.spines['bottom'].set_color('black')
    
    # 统一刻度线宽度和长度，缩小X轴刻度标签的padding
    ax.tick_params(axis='x', colors='black', width=UNIFIED_LINEWIDTH, length=5, pad=2)
    ax.tick_params(axis='y', width=UNIFIED_LINEWIDTH, length=5)
    
    plt.tight_layout()
    
    # 保存
    policy_short = "Electricity_NMBI" if "Electricity" in policy_name else "Climate_Cooperation"
    out_path = output_dir / f"{policy_short}_ridge_plot.png"
    fig.savefig(out_path, dpi=300, bbox_inches='tight', pad_inches=0.15)
    plt.close()
    
    print(f"  ✓ 山脊图已保存: {out_path.name} (包含 {valid_idx} 个国家)")

# 主循环：为两个政策生成山脊图
print(f"\n开始生成山脊图...\n")
print(f"选定的7个国家:")
for country_code, continent in selected_countries_info:
    print(f"  • {country_code} ({continent})")
print()

for policy_idx, (policy_name, policy_label) in enumerate(policies.items(), 1):
    print(f"\n{'='*70}")
    print(f"[{policy_idx}/{len(policies)}] 政策: {policy_name}")
    print(f"{'='*70}\n")
    
    plot_ridge_chart(policy_name, policy_label, selected_countries_info, df)

print(f"\n{'='*70}")
print(f"✅ 完成！共生成 {len(policies)} 个山脊图")
print(f"📁 保存位置: {output_dir}")
print(f"{'='*70}")



开始生成山脊图...

选定的7个国家:
  • CAN (North America)
  • DEU (Europe)
  • CHN (Asia)
  • ARG (South America)
  • AUS (Oceania)
  • ZAF (Africa)
  • JPN (Asia)


[1/2] 政策: Electricity – non market-based instruments



C:\Users\28274\AppData\Local\Temp\ipykernel_22740\4191798953.py:65: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('coolwarm')


  ✓ 山脊图已保存: Electricity_NMBI_ridge_plot.png (包含 7 个国家)

[2/2] 政策: International climate co-operation



C:\Users\28274\AppData\Local\Temp\ipykernel_22740\4191798953.py:65: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('coolwarm')


  ✓ 山脊图已保存: Climate_Cooperation_ridge_plot.png (包含 7 个国家)

✅ 完成！共生成 2 个山脊图
📁 保存位置: f:\Desktop\CAMPF_Supplementary\data\2-2-Country_Policy_Trends
